In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten, Input
from tensorflow.keras.datasets import mnist
from keras import Sequential

import warnings
import os
warnings.simplefilter('ignore')
os.close(2)
os.open(os.devnull, os.O_RDWR)

2023-10-08 00:53:30.970609: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-08 00:53:31.253735: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2

# Model creation

In [3]:
class ObjectDetection(tf.keras.Model):
    def __init__(self,n_class):
        super(ObjectDetection,self).__init__()
        self.n_class=n_class
        self.base_model=Sequential([
            Conv2D(filters=64,kernel_size=3,padding='same',strides=1,activation='relu'),
            BatchNormalization(),
            MaxPool2D(pool_size=2,strides=2),
            Conv2D(filters=128,kernel_size=3,padding='same',strides=1,activation='relu'),
            BatchNormalization(),
            Conv2D(filters=128,kernel_size=3,padding='same',strides=1,activation='relu'),
            BatchNormalization()
        ])
        
        self.flatten =Flatten()
        
        self.regressor = Sequential([
    		Dense(units=128, activation='relu'),
    		Dense(units=64,activation='relu'),
    		Dense(units=32,activation='relu'),
    		Dense(units=4,activation='sigmoid')]
        )
        
        self.classifier = Sequential([
			Dense(units=256,activation='relu'),
			Dropout(0.2),
			Dense(units=256, activation='relu'),
			Dropout(0.2),
			Dense(units=self.n_class)]
        )

    def call(self,inputs):
        x=self.base_model(inputs)
        x=self.flatten(x)
        regressor=self.regressor(x)
        classifier=self.classifier(x)
        return (regressor,classifier)

In [4]:
input_tensor = Input(shape=(224, 224, 3))
model = ObjectDetection(n_class=10)
output=model(input_tensor)
model.